In [1]:
import warnings 
warnings.filterwarnings('ignore')

## WIP - Skjønne hvordan CNN bygges opp. 
Basert på: 
* https://cv-tricks.com/tensorflow-tutorial/training-convolutional-neural-network-for-image-classification/ 
* https://www.tensorflow.org/tutorials/estimators/cnn#building_the_cnn_mnist_classifier 

In [2]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
# Just to reset the graph such that it is possible to rerun this in a
# Jupyter cell without resetting the whole kernel.
tf.reset_default_graph()

In [3]:
from keras.datasets import mnist
import matplotlib.pyplot as plt

Using TensorFlow backend.


In [4]:


from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

# Imports
import numpy as np
import tensorflow as tf

#tf.logging.set_verbosity(tf.logging.INFO)
tf.logging.set_verbosity(tf.logging.ERROR)


# Our application logic will be added here



In [5]:
def create_weights(shape):
    return tf.Variable(tf.truncated_normal(shape, stddev=0.05))

def create_biases(size):
    return tf.Variable(tf.constant(0.05, shape=[size]))



def create_convolutional_layer(input,
               num_input_channels, 
               conv_filter_size,        
               num_filters):  
    
    ## We shall define the weights that will be trained using create_weights function.
    weights = create_weights(shape=[conv_filter_size, conv_filter_size, num_input_channels, num_filters])
    ## We create biases using the create_biases function. These are also trained.
    biases = create_biases(num_filters)

    ## Creating the convolutional layer
    layer = tf.nn.conv2d(input=input,
                     filter=weights,
                     strides=[1, 1, 1, 1],
                     padding='SAME')

    layer += biases

    ## We shall be using max-pooling.  
    layer = tf.nn.max_pool(value=layer,
                            ksize=[1, 2, 2, 1],
                            strides=[1, 2, 2, 1],
                            padding='SAME')
    ## Output of pooling is fed to Relu which is the activation function for us.
    layer = tf.nn.relu(layer)

    return layer


def create_flatten_layer(layer):
    #We know that the shape of the layer will be [batch_size img_size img_size num_channels] 
    # But let's get it from the previous layer.
    layer_shape = layer.get_shape()

    ## Number of features will be img_height * img_width* num_channels. But we shall calculate it in place of hard-coding it.
    num_features = layer_shape[1:4].num_elements()

    ## Now, we Flatten the layer so we shall have to reshape to num_features
    layer = tf.reshape(layer, [-1, num_features])

    return layer


def create_fc_layer(input,          
             num_inputs,    
             num_outputs,
             use_relu=True):
    
    #Let's define trainable weights and biases.
    weights = create_weights(shape=[num_inputs, num_outputs])
    biases = create_biases(num_outputs)

    # Fully connected layer takes input x and produces wx+b.Since, these are matrices, we use matmul function in Tensorflow
    layer = tf.matmul(input, weights) + biases
    if use_relu:
        layer = tf.nn.relu(layer)

    return layer

In [6]:
def show_progress(epoch, feed_dict_train, feed_dict_validate, val_loss):
    acc = session.run(accuracy, feed_dict=feed_dict_train)
    val_acc = session.run(accuracy, feed_dict=feed_dict_validate)
    msg = "Training Epoch {0} --- Training Accuracy: {1:>6.1%}, Validation Accuracy: {2:>6.1%},  Validation Loss: {3:.3f}"
    print(msg.format(epoch + 1, acc, val_acc, val_loss))

In [7]:
def next_batch(num, data, labels):
    
    idx = np.arange(0 , len(data))
    np.random.shuffle(idx)
    idx = idx[:num]
    #print(idx)
    #data_shuffle = [data[ i] for i in idx]
    data_shuffle = data[idx] 
    #print(data_shuffle[0])

    #print(data_shuffle.shape)
    labels_shuffle = labels[idx]

    #labels_shuffle = [labels[ i] for i in idx]

    return np.asarray(data_shuffle), np.asarray(labels_shuffle)

In [8]:
def train(num_iteration):
    global total_iterations
    
    for i in range(total_iterations, total_iterations + num_iteration):
        #images_train = images_train.reshape(1, img_size,img_size,num_channels)

        x_batch, y_true_batch = next_batch(batch_size, images_train, labels_train)
        #print(images_train)
        x_valid_batch, y_valid_batch = next_batch(batch_size, images_test, labels_test)

        
        feed_dict_tr = {x: x_batch,
                           y_true: y_true_batch}
        
        
        #print(feed_dict_tr)
        
        feed_dict_val = {x: x_valid_batch,
                              y_true: y_valid_batch}

        #print(feed_dict_val) 
        
        session.run(optimizer, feed_dict=feed_dict_tr)

        if i % int(data.train.num_examples/batch_size) == 0: 
            val_loss = session.run(cost, feed_dict=feed_dict_val)
            epoch = int(i / int(data.train.num_examples/batch_size))    
            
            show_progress(epoch, feed_dict_tr, feed_dict_val, val_loss)
            saver.save(session, 'mnistmodel') 


    total_iterations += num_iteration

In [9]:
# Load training and eval data 

(images_train, labels_train),(images_test, labels_test) = mnist.load_data()
# Convert the samples from integers to floating-point numbers:
images_train, images_test = images_train / 255.0, images_test / 255.0
#print(images_train[0].shape)
# Konverterer labels til rett format

labels_train = np.asarray(labels_train, dtype=np.int32)     
labels_test = np.asarray(labels_test, dtype=np.int32)
#print()

[nr_train , imgh, imgw] = images_train.shape
[nr_test , imgh, imgw] = images_test.shape

In [10]:

import tensorflow as tf
import time
from datetime import timedelta
import math

import random
import numpy as np
import os
tf.reset_default_graph()

#Adding Seed so that random initialization is consistent
from numpy.random import seed
seed(1)
from tensorflow import set_random_seed
set_random_seed(2)


batch_size = 32

#Prepare input data
classes = ['0','1','2','3','4','5','6','7','8','9']
num_classes = len(classes)

# 20% of the data will automatically be used for validation
validation_size = 0.2
img_size = 28
num_channels = 1
#train_path='training_data'

# We shall load all the training and validation images and labels into memory using openCV and use that during training
#data = dataset.read_train_sets(train_path, img_size, classes, validation_size=validation_size)


#print("Complete reading input data. Will Now print a snippet of it")
#print("Number of files in Training-set:\t\t{}".format(len(data.train.labels)))
#print("Number of files in Validation-set:\t{}".format(len(data.valid.labels)))



session = tf.Session()

x = tf.placeholder(tf.float32, shape=[None, img_size,img_size], name='x')

## labels
y_true = tf.placeholder(tf.float32, shape=[None, num_classes], name='y_true')
y_true_cls = tf.argmax(y_true, dimension=1)



##Network graph params
filter_size_conv1 = 3 
num_filters_conv1 = 32

filter_size_conv2 = 3
num_filters_conv2 = 32

filter_size_conv3 = 3
num_filters_conv3 = 64
    
fc_layer_size = 28

input_layer = tf.reshape(x, [28,28,1,-1])
#y_true = tf.reshape(y_true, [-1])

layer_conv1 = create_convolutional_layer(input=input_layer,
               num_input_channels=num_channels,
               conv_filter_size=filter_size_conv1,
               num_filters=num_filters_conv1)
print(layer_conv1)
#layer_conv1 = create_convolutional_layer(input=x,
#               num_input_channels=num_channels,
#               conv_filter_size=filter_size_conv1,
#               num_filters=num_filters_conv1)

layer_conv2 = create_convolutional_layer(input=layer_conv1,
               num_input_channels=num_filters_conv1,
               conv_filter_size=filter_size_conv2,
               num_filters=num_filters_conv2)

layer_conv3= create_convolutional_layer(input=layer_conv2,
               num_input_channels=num_filters_conv2,
               conv_filter_size=filter_size_conv3,
               num_filters=num_filters_conv3)
          
layer_flat = create_flatten_layer(layer_conv3)
print(layer_flat)
layer_fc1 = create_fc_layer(input=layer_flat,
                     num_inputs=layer_flat.get_shape()[1:4].num_elements(),
                     num_outputs=fc_layer_size,
                     use_relu=True)
print(layer_fc1)
layer_fc2 = create_fc_layer(input=layer_fc1,
                     num_inputs=fc_layer_size,
                     num_outputs=num_classes,
                     use_relu=False) 

print(layer_fc2)
y_pred = tf.nn.softmax(layer_fc2,name='y_pred')
print(y_pred)
y_pred_cls = tf.argmax(y_pred, dimension=1)

#print(y_pred)
session.run(tf.global_variables_initializer())
#cross_entropy = tf.nn.softmax_cross_entropy_with_logits(logits=layer_fc2,
#                                                    labels=y_true)

#print(layer_fc2)
cross_entropy = tf.nn.sigmoid_cross_entropy_with_logits(logits=layer_fc2,
                                                    labels=y_true)
print(cross_entropy)
#cross_entropy = tf.nn.sigmoid_cross_entropy_with_logits(logits=y_pred,
#                                                    labels=y_true)
cost = tf.reduce_mean(cross_entropy)
optimizer = tf.train.AdamOptimizer(learning_rate=1e-4).minimize(cost)
correct_prediction = tf.equal(y_pred_cls, y_true_cls)
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

session.run(tf.global_variables_initializer()) 

total_iterations = 0

saver = tf.train.Saver()

train(num_iteration=3000)

Tensor("Relu:0", shape=(28, 14, 1, 32), dtype=float32)
Tensor("Reshape_1:0", shape=(28, 256), dtype=float32)
Tensor("Relu_3:0", shape=(28, 28), dtype=float32)
Tensor("add_4:0", shape=(28, 10), dtype=float32)
Tensor("y_pred:0", shape=(28, 10), dtype=float32)
Tensor("logistic_loss:0", shape=(28, 10), dtype=float32)


ValueError: Cannot feed value of shape (32,) for Tensor 'y_true:0', which has shape '(?, 10)'